Run the notebook at TODAY after 15:00:00 (so the market is closed and data is available).

Input your current position.

The orders are for tomorrow.

In [16]:
TODAY = '2024-12-09'
# NEXTDAY = '2024-12-10'
# PHDAY = '2024-10-16' # place holder

test_split = (TODAY, TODAY)
pos_txt = f'''\
date {TODAY}
cash 100000.00
SH601698 200
'''

REC_LIST = [
    ('EXP_BENCH', '11ef0b136a7d4930a130d99c9437a8a3') # GBDT BENCH_NOW 
]

EXP_NAME, rid = REC_LIST[0]
TOPK = 5
NDROP = 1
HT = 0

info = {
    'ALGO': ['GBDT'],
    'market': ['csi300_ext'], 
    'benchmark': ["SH000300"], 
    'feat': ["Alpha360"], 
    'label': ['r0'],
    'params': [f'topk {TOPK} ndrop {NDROP} HT {HT}']
}

In [17]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

[2037073:MainThread](2024-12-10 17:06:18,075) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[2037073:MainThread](2024-12-10 17:06:18,079) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[2037073:MainThread](2024-12-10 17:06:18,080) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [18]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
print(dt)
print(account)
print(norm_pos)

2024-12-09 09:00:00
{'SH601698': {'amount': 200}, 'cash': 100000.0}
{'_settle_type': 'None', 'position': {'SH601698': {'amount': 777, 'price': 5.550973415374756}, 'cash': 100000.0, 'now_account_value': 104313.10634374619}, 'init_cash': 100000.0}


Prepare dataset

In [19]:
nameDFilter = NameDFilter(name_rule_re='(SH60[0-9]{4})|(SZ00[0-9]{4})')
filter_pipe=[nameDFilter]
benchmark = BENCH_Step(test_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

In [20]:
dataset = init_instance_by_config(benchmark.dataset_config)
dataset.prepare('test')

[2037073:MainThread](2024-12-10 17:07:17,108) INFO - qlib.timer - [log.py:127] - Time cost: 50.223s | Loading data Done
/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/qlib/utils/paral.py:54: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  return df.groupby(axis=axis, level=level).apply(apply_func)
[2037073:MainThread](2024-12-10 17:07:18,186) INFO - qlib.timer - [log.py:127] - Time cost: 1.075s | ProcessInf Done
[2037073:MainThread](2024-12-10 17:07:18,220) INFO - qlib.timer - [log.py:127] - Time cost: 0.032s | ZScoreNorm Done
[2037073:MainThread](2024-12-10 17:07:18,233) INFO - qlib.timer - [log.py:127] - Time cost: 0.012s | Fillna Done
[2037073:MainThread](2024-12-10 17:07:18,248) INFO - qlib.timer - [log.py:127] - Time cost: 0.003s | DropnaLabel Done
[2037073:MainThread](2024-12-10 17:07:18,256) INFO - qlib.timer - [log.py:127] - Time cost: 0.006s | CSZScoreNorm Done
[2037073:MainThread](2024-12-10 17:07:1

CLOSE59   CLOSE58   CLOSE57   CLOSE56  \
datetime   datetime   instrument                                           
2024-12-09 2024-12-09 SH600000    0.380261  0.433604  0.553535  0.364520   
                      SH600009    0.715424  0.671328  0.573179  0.578115   
                      SH600010   -1.209137 -1.185183 -1.226270 -1.317633   
                      SH600011    0.871178  0.892797  0.887575  0.686077   
                      SH600015    0.032989  0.126529  0.167633 -0.226323   
...                                    ...       ...       ...       ...   
                      SZ002841   -0.276334 -0.195780  0.039904  0.027124   
                      SZ002916    0.895764  0.998220  1.129236  1.095255   
                      SZ002920   -0.256835 -0.769004 -0.595419 -0.505888   
                      SZ002938    1.201013  1.262861  1.276212  1.229649   
                      SZ003816    1.866625  1.952075  1.827291  1.466104   

                                   CLOSE55   CLOSE54   CLOSE53   CLOSE52  \
datetime   datetime   instrument                                           
2024-12-09 2024-12-09 SH600000    0.619462  0.527778  0.444348  0.564344   
                      SH600009    0.569609  0.548440  0.352347  0.447351   
                      SH600010   -1.286217 -1.157765 -1.197092 -1.224125   
                      SH600011    0.748245  0.795325  0.755529  0.821076   
                      SH600015   -0.217552 -0.227477 -0.209413 -0.300309   
...                                    ...       ...       ...       ...   
                      SZ002841   -0.037979 -0.070686 -0.102262 -0.276117   
                      SZ002916    1.103953  1.473245  1.479820  0.888071   
                      SZ002920   -0.679318 -0.637769 -0.356145 -0.489468   
                      SZ002938    1.118549  1.522215  0.872022  0.950540   
                      SZ003816    1.384319  1.501125  1.432976  1.388226   

                                   CLOSE51   CLOSE50  ...   VOLUME8   VOLUME7  \
datetime   datetime   instrument                      ...                       
2024-12-09 2024-12-09 SH600000    0.538619  0.612362  ...  0.371701  0.519940   
                      SH600009    0.461121  0.429895  ... -0.435452 -0.517434   
                      SH600010   -1.260954 -1.217895  ... -0.791058 -1.086603   
                      SH600011    0.699275  0.895552  ... -0.989470 -1.071738   
                      SH600015   -0.281201 -0.026631  ...  0.493084  0.066278   
...                                    ...       ...  ...       ...       ...   
                      SZ002841   -0.347254 -0.636459  ... -1.034430 -1.171517   
                      SZ002916    0.743308  0.549785  ...  1.116386  0.337578   
                      SZ002920   -0.423971 -0.374866  ... -0.520345 -1.107765   
                      SZ002938    0.947460  0.759194  ...  3.849713  1.739356   
                      SZ003816    1.360673  1.548283  ...  0.053501  0.184228   

                                   VOLUME6   VOLUME5   VOLUME4   VOLUME3  \
datetime   datetime   instrument                                           
2024-12-09 2024-12-09 SH600000    0.513631  0.247665  0.273474  1.013062   
                      SH600009   -0.683635 -0.125338 -1.014874 -0.842902   
                      SH600010   -1.425927 -1.007972 -1.215486 -0.878596   
                      SH600011   -1.020599 -1.182339 -1.201688 -1.338477   
                      SH600015    1.819173  1.277319  1.397452  1.288029   
...                                    ...       ...       ...       ...   
                      SZ002841   -1.088700 -1.100294 -1.349712 -1.172823   
                      SZ002916    0.294552  1.324426  1.391329  0.362786   
                      SZ002920   -0.719101 -0.022776 -0.913619 -0.928149   
                      SZ002938    0.757538  2.512988  1.196237  1.747236   
                      SZ003816   -0.321717 -0.072198  1.553722  0.576619   

              

Load and run model

In [21]:
with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
    recorder = R.get_recorder(recorder_id=rid)
    model = recorder.load_object("trained_model")

[2037073:MainThread](2024-12-10 17:08:34,750) INFO - qlib.workflow - [exp.py:258] - Experiment 2 starts running ...
[2037073:MainThread](2024-12-10 17:08:34,759) INFO - qlib.workflow - [recorder.py:341] - Recorder 31f52ed1f88c411c885de9715577a98e starts running under Experiment 2 ...
[2037073:MainThread](2024-12-10 17:08:36,629) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


In [22]:
EXP_NAME = 'realworld_position_maker'

# backtest and analysis
with R.start(experiment_name=EXP_NAME, uri='./mlrun'):    
    recorder = R.get_recorder()
    ba_rid = R.get_recorder().id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

[2037073:MainThread](2024-12-10 17:08:47,084) WARNING - qlib.workflow - [expm.py:230] - No valid experiment found. Create a new experiment with name realworld_position_maker.
[2037073:MainThread](2024-12-10 17:08:47,092) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[2037073:MainThread](2024-12-10 17:08:47,100) INFO - qlib.workflow - [recorder.py:341] - Recorder 653b3efd9a0e4ac5b3401c7e81c91aa6 starts running under Experiment 3 ...
[2037073:MainThread](2024-12-10 17:08:47,267) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 3
[2037073:MainThread](2024-12-10 17:08:47,275) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


'The following are prediction results of the LGBModel model.'
                                     score
datetime   datetime   instrument          
2024-12-09 2024-12-09 SH600000    0.050204
                      SH600009    0.001767
                      SH600010   -0.118445
                      SH600011   -0.012411
                      SH600015   -0.140165


In [26]:
with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
    recorder = R.get_recorder(recorder_id=ba_rid)
    pred_df = recorder.load_object("pred.pkl")

[2037073:MainThread](2024-12-10 17:10:59,564) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[2037073:MainThread](2024-12-10 17:10:59,582) INFO - qlib.workflow - [recorder.py:341] - Recorder 41d4c81502004979822165a208095d1b starts running under Experiment 3 ...
[2037073:MainThread](2024-12-10 17:10:59,814) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


In [27]:
rank_df = pred_df.droplevel([0,1]).sort_values(by='score', ascending=False)

Top / bottom predictions

In [43]:
rank_df.head(20)

,score
instrument,
SH600989,0.544234
SH600732,0.218179
SH600132,0.196683
SZ002371,0.172311
SH603486,0.171904
SH600025,0.150001
SZ002459,0.135651
SH601698,0.135534
SH600547,0.134165


In [29]:
# rank_df.tail(10).T

Strategy preparation

In [30]:
strategy_config = {
    "class": "TopkDropoutStrategy",
    "module_path": "lilab.qlib.strategy.signal_strategy",
    "kwargs": {
        "model": model,
        "dataset": dataset,
        "topk": TOPK,
        "n_drop": NDROP,
        "hold_thresh": HT,
        "signal": pred_df
    },
}

port_analysis_config = {
    "executor": benchmark.executor_config,
    "strategy": strategy_config,
    "backtest": benchmark.backtest_config
}

In [31]:
_norm_pos = norm_pos.__dict__['position']
_norm_pos

{'SH601698': {'amount': 777, 'price': 5.550973415374756},
 'cash': 100000.0,
 'now_account_value': 104313.10634374619}

In [32]:
from qlib.backtest import get_strategy_executor
trade_strategy, trade_executor = \
    get_strategy_executor(TODAY, TODAY, strategy_config, benchmark.executor_config, account=_norm_pos)
trade_executor.reset(start_time=TODAY, end_time=TODAY)
trade_strategy.reset(level_infra=trade_executor.get_level_infra())

[2037073:MainThread](2024-12-10 17:12:03,526) INFO - qlib.backtest caller - [__init__.py:93] - Create new exchange
[2037073:MainThread](2024-12-10 17:12:09,483) WARNING - qlib.data - [data.py:665] - load calendar error: freq=day, future=True; return current calendar!
[2037073:MainThread](2024-12-10 17:12:09,487) WARNING - qlib.data - [data.py:668] - You can get future calendar by referring to the following document: https://github.com/microsoft/qlib/blob/main/scripts/data_collector/contrib/README.md
[2037073:MainThread](2024-12-10 17:12:09,507) WARNING - qlib.BaseExecutor - [executor.py:121] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor object at 0x7fd1d7542d00>


In [33]:
dt_lst = sorted(list(pos_history.keys()))[::-1]
for k, v in _norm_pos.items():
    if not isinstance(v, dict):
        continue
    cnt = 1
    for dt in dt_lst:
        tmp_pos = pos_history[dt]
        if k not in tmp_pos.get_stock_list():
            break
        cnt += 1
    print(k, cnt)
    trade_strategy.trade_position.update_stock_count(k, 'day', cnt)

SH601698 2


In [34]:
pred_df0 = pred_df.droplevel([0,1])
# pred_df0
sell_list, buy_list = trade_strategy.generate_trade_decision(pred_score=pred_df0, today=TODAY)

# Sell Order 

In [35]:
df = pd.DataFrame([[order.stock_id, order.amount] for order in sell_list], columns=['id', 'norm_amount']).set_index('id')
stock_list = df.index.values
fields = ['$close', '$factor', '$close/$factor', \
          '100*(Ref($close, -1) / $close - 1)', '100*(Ref($close, -2) / Ref($close, -1) - 1)']
_df = D.features(stock_list, fields, start_time=pd.to_datetime(TODAY)-pd.to_timedelta(5, 'day'), end_time=TODAY, freq='day')
_df.columns = ['norm_close', 'factor', 'close', 'r0', 'r1']
df_sell_hist = _df


In [36]:
fields = ['$close', '$factor', '$close/$factor']
_df = D.features(stock_list, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['norm_close', 'factor', 'close']
_df = _df.droplevel(1)
df = pd.concat([df,_df],axis=1,join='inner')
df['amount'] = df['norm_amount'] * df['factor']
df['receive'] = df['amount'] * df['close']
df['amount'] = df['amount'].apply(lambda x: round(x/100)*100)
df = df.loc[df['amount'] > 0, :]
df

,norm_amount,norm_close,factor,close,amount,receive
SH601698,777,5.550973,0.257347,21.570002,200,4313.106527


In [37]:
print(df['receive'].sum())

4313.10652749158


# Buy Order

In [38]:
df = pd.DataFrame([[order.stock_id, order.amount] for order in buy_list], columns=['id', 'norm_amount']).set_index('id')
stock_list = df.index.values
fields = ['$close', '$factor', '$close/$factor', \
          '100*(Ref($close, -1) / $close - 1)', '100*(Ref($close, -2) / Ref($close, -1) - 1)']
_df = D.features(stock_list, fields, start_time=pd.to_datetime(TODAY)-pd.to_timedelta(5, 'day'), end_time=TODAY, freq='day')
_df.columns = ['norm_close', 'factor', 'close', 'r0', 'r1']
df_buy_hist = _df

In [39]:
fields = ['$close', '$factor', '$close/$factor']
_df = D.features(stock_list, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['norm_close', 'factor', 'close']
_df = _df.droplevel(1)
df = pd.concat([df,_df],axis=1,join='inner')
df['amount'] = df['norm_amount'] * df['factor']
df['pay'] = df['amount'] * df['close']
df['amount'] = df['amount'].apply(lambda x: round(x/100)*100)
df = df.loc[df['amount'] > 0, :]
df

,norm_amount,norm_close,factor,close,amount,pay
SH600989,15357.983042,1.187526,0.071624,16.579998,1100,18237.997818
SH600732,5255.528462,3.609532,0.266386,13.550000,1400,18970.000267
SH600132,5358.965806,3.561508,0.055981,63.619999,300,19085.999680
SH603486,7907.838014,2.501822,0.050583,49.460003,400,19784.001160


In [40]:
print(df['pay'].sum())

76077.99892425537


Check

In [41]:
df_sell_hist

norm_close    factor      close        r0        r1
instrument datetime                                                       
SH601698   2024-12-04    5.692514  0.257347  22.120001  2.169979 -1.371682
           2024-12-05    5.816041  0.257347  22.600002 -1.371682 -3.230149
           2024-12-06    5.736263  0.257347  22.290001 -3.230149       NaN
           2024-12-09    5.550973  0.257347  21.570002       NaN       NaN

In [42]:
df_buy_hist

norm_close    factor       close        r0        r1
instrument datetime                                                        
SH600132   2024-12-04    3.595097  0.055981   64.220001  2.024293 -0.824177
           2024-12-05    3.667872  0.055981   65.520004 -0.824177 -2.092957
           2024-12-06    3.637642  0.055981   64.980003 -2.092957       NaN
           2024-12-09    3.561508  0.055981   63.619999       NaN       NaN
SH600732   2024-12-04    4.067717  0.266386   15.270000 -1.178789 -4.174948
           2024-12-05    4.019767  0.266386   15.089999 -4.174948 -6.293220
           2024-12-06    3.851944  0.266386   14.460000 -6.293220       NaN
           2024-12-09    3.609532  0.266386   13.550000       NaN       NaN
SH600989   2024-12-04    1.131659  0.071624   15.799998 -0.443029  0.953579
           2024-12-05    1.126645  0.071624   15.730000  0.953579  4.408062
           2024-12-06    1.137389  0.071624   15.879998  4.408062       NaN
           2024-12-09    1.187526  0.071624   16.579998       NaN       NaN
SH603486   2024-12-04    2.548864  0.050583   50.390003 -0.694585  1.638687
           2024-12-05    2.531160  0.050583   50.040001  1.638687 -2.752656
           2024-12-06    2.572638  0.050583   50.860004 -2.752656       NaN
           2024-12-09    2.501822  0.050583   49.460003       NaN       NaN
SZ002371   2024-12-04   32.554695  0.080871  402.549988 -0.881869  1.651621
           2024-12-05   32.267605  0.080871  399.000000  1.651621 -1.920658
           2024-12-06   32.800545  0.080871  405.589996 -1.920658       NaN
           2024-12-09   32.170559  0.080871  397.800018       NaN       NaN